In [ ]:
# !pip install --upgrade bigqueue-client

In [10]:
from bigqueue.client.client import BigqueueClient, BigqueueClientConfig

In [11]:
from bigqueue.client.entity import *

In [12]:
config = {
    'bigqueue_service_url': 'https://bigqueue.a.musta.ch',
    'bigqueue_service_version': '0',
    'datastore_type': 'S3',
    'datastore_init_kwargs': {
        'bucket': 'airbnb-bigqueue',
        'base_prefix': 'staging'
    }
}

In [13]:
bigq_client = BigqueueClient(BigqueueClientConfig(**config))

In [14]:
model = bigq_client.upload_model(
    owner="shaowei_su",
    env="devel/shaowei/rl:0.0.5",
    model_file_root='/home/tianbing_xu/repos/GRPO/TinyZero',
    model_file_path='verl/trainer/main_ppo.py',
    name='Demo Ray Job Codebase for MATH RL',
    flavor='ray')

In [ ]:
job = bigq_client.create_ray_job(
    model_id=model['id'],
    name="REINFORCE MATH DIFFICULT: base Qwen2.5-Math-7B",
    owner='tianbing_xu',
    team='one_airbnb_team_id_2934',
    queue='csp-mle-p4',
    execution_config={
        'head': {'cpu': 12, 'memory': '20Gi',  'ephemeral_storage': '50Gi',},
        'worker': {'number': 1, 'gpu': 4, 'accelerator': 'nvidia-tesla-a100xl', 'cpu': 24, 'memory': '240Gi',  'ephemeral_storage': '200Gi',}
    },
    runtime_env={"working_dir": "./", "env_vars": {"WANDB_API_KEY": "09f781a401d2ec1d3ff5f9b550eb1ec0f291fb31", "HF_ENDPOINT": "http://bigqueue-service-production.bigqueue-service-production:6580/hfproxy"}},
    entrypoint="export WANDB_API_KEY=\"09f781a401d2ec1d3ff5f9b550eb1ec0f291fb31\" && export N_GPUS=4 && export BASE_MODEL=Qwen/Qwen2.5-Math-7B && export DATA_DIR=./data/simplelr_qwen_level3to5 && export ROLLOUT_TP_SIZE=2 && export EXPERIMENT_NAME=reinforce_qwen_7b_math && export VLLM_ATTENTION_BACKEND=XFORMERS && python3 -m verl.trainer.main_ppo algorithm.adv_estimator=reinforce_plus_plus data.train_files=$DATA_DIR/train.parquet data.val_files=$DATA_DIR/test.parquet data.train_batch_size=32 data.val_batch_size=32 data.max_prompt_length=1024 data.max_response_length=2048 actor_rollout_ref.model.path=$BASE_MODEL actor_rollout_ref.actor.optim.lr=1e-6 actor_rollout_ref.model.use_remove_padding=True actor_rollout_ref.actor.ppo_mini_batch_size=32 actor_rollout_ref.actor.ppo_micro_batch_size=4 actor_rollout_ref.actor.clip_ratio=0.2 actor_rollout_ref.actor.use_kl_loss=True actor_rollout_ref.actor.kl_loss_coef=0.0001 actor_rollout_ref.actor.kl_loss_type=low_var_kl actor_rollout_ref.rollout.temperature=1.0 actor_rollout_ref.model.enable_gradient_checkpointing=True actor_rollout_ref.actor.fsdp_config.param_offload=False actor_rollout_ref.actor.fsdp_config.optimizer_offload=False actor_rollout_ref.rollout.log_prob_micro_batch_size=4 actor_rollout_ref.rollout.tensor_model_parallel_size=$ROLLOUT_TP_SIZE actor_rollout_ref.rollout.name=vllm actor_rollout_ref.rollout.gpu_memory_utilization=0.6 actor_rollout_ref.rollout.n=8 actor_rollout_ref.ref.log_prob_micro_batch_size=4 actor_rollout_ref.ref.fsdp_config.param_offload=True algorithm.kl_ctrl.kl_coef=0.001 trainer.logger=\"['console','wandb']\" trainer.project_name=MathRLZero trainer.experiment_name=$EXPERIMENT_NAME trainer.n_gpus_per_node=$N_GPUS trainer.nnodes=1 trainer.save_freq=-1 trainer.test_freq=5 trainer.total_epochs=1 trainer.total_training_steps=100",
)

In [16]:
job

{'id': 'b4f3ab5e-8379-465d-bba2-d92107703f46'}